In [1]:
# Import os to set API key
import os
# Import OpenAI as main LLM service
from langchain.chat_models import ChatOpenAI #llm
from langchain.embeddings import OpenAIEmbeddings


#memory
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# Import PDF document loaders...there's other ones as well!
from langchain.document_loaders import PyPDFLoader
# Import chroma as the vector store 
from langchain.vectorstores import Chroma

from langchain.agents import OpenAIFunctionsAgent
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter


In [2]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, LLMChain

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

In [4]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)

need to use a document, not strings
https://python.langchain.com/docs/modules/agents/how_to/agent_vectorstore

In [5]:
from apikey import apikey
# Set APIkey for OpenAI Service
# Can sub this out for other LLM providers
os.environ['OPENAI_API_KEY'] = apikey

# Create instance of OpenAI LLM
llm = ChatOpenAI(temperature=0.1,model="gpt-3.5-turbo-0613")
embeddings = OpenAIEmbeddings()

In [6]:
## Create and load PDF Loader
loader = PyPDFLoader('Resume2023Jul.pdf')
## Split pages from pdf 
#pages = loader.load_and_split()
## Load documents into vector database aka ChromaDB
#store = Chroma.from_documents(pages, embeddings, collection_name='annualreport')

In [10]:
#vectorstore_info = VectorStoreInfo(
#    name="resume",
#    description="resume pdf",
#    vectorstore=store
#)

In [11]:
#toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)

In [9]:
test_string = """Jay Chou (simplified Chinese: 周杰伦; traditional Chinese: 周杰倫; pinyin: Zhōu Jiélún; born January 18, 1979)[1] is a Taiwanese singer, songwriter, record producer, rapper, actor, television personality, and businessman. Dubbed the "King of Mandopop", and having sold over 30 million records, Chou is one of the best-selling artists in Taiwan and is known for his work with lyricist Vincent Fang, with whom he has frequently collaborated on his music.[2]

In 2000, Chou released his debut studio album, Jay (2000), under the record company Alfa Music. Chou rose to fame with the release of his second studio album, Fantasy (范特西) (2001), which combined Western and Eastern music styles. The album won five Golden Melody Awards, including Album of the Year. He has since further released twelve more studio albums, spawning a string of hit singles and gaining significant prominence in Asian communities such as Taiwan, Hong Kong SAR, China Mainland, Malaysia, and Singapore. Chou has embarked on six world tours, performing in cities around the world to more than 10 million spectators as of 2019.[3]

In 2007, Chou established his own record and management company JVR Music.[4] Outside of music, Chou has served as the President of his own fashion brand PHANTACi since 2006. As an actor, Chou made his acting debut in the film Initial D 頭文字D(2005), followed shortly by a starring role in the epic Curse of the Golden Flower (2006). He has since starred in a number of movies, becoming known to Western audiences when he made his Hollywood debut in 2011 with The Green Hornet, starring alongside Seth Rogen and Christoph Waltz, followed by Now You See Me 2 (2016)."""

In [7]:
#use this
from langchain.document_loaders import TextLoader

#loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [9]:
#use this
from langchain.chains import RetrievalQA
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

In [10]:
tools = [
    Tool(
        name="resume",
        func=ruff.run,
        description="test",
    ),
]

In [11]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a chatbot having a conversation with a human."), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injectd
])
    
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [12]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [13]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [14]:
agent.run("what are the personal projects?")



> Entering new  chain...


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


I need to find the section in the resume that lists personal projects.
Action: resume
Action Input: test
Observation: I'm sorry, I don't understand what you mean by "test." Can you please provide more information or ask a specific question?
Thought:

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


I need to clarify the question or provide more information.
Action: resume
Action Input: test
Observation: I'm sorry, I don't understand what you mean by "test." Can you please provide more information or clarify your question?
Thought:

...

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1



Observation: I'm sorry, I don't understand what you mean by "test." Can you please provide more information or ask a specific question?
Thought:

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


I need to ask for clarification or more information.
Action: resume
Action Input: test
Observation: I'm sorry, I don't understand what you mean by "test." Can you please provide more information or clarify your question?
Thought:

OutputParserException: Could not parse LLM output: `I now know that the input question is unclear and requires more information or clarification.`